In [0]:
# Importamos la función para obtener la fecha/hora actual
from pyspark.sql.functions import current_timestamp

# ============================================================
# 1. Lectura y consolidación de datos desde múltiples sucursales
# ============================================================
# Usamos Spark SQL para leer las tablas de ventas de cada sucursal
# y unificarlas en un solo DataFrame con UNION ALL.
# Además, añadimos la columna 'archivo_fuente' usando _metadata.file_path
# para mantener trazabilidad del archivo origen de cada registro.

df_TechMart = spark.sql(
    """
    SELECT *, _metadata.file_path AS archivo_fuente
    FROM volumen_ventas.default.ventas_alajuela
    UNION ALL
    SELECT *, _metadata.file_path AS archivo_fuente
    FROM volumen_ventas.default.ventas_cartago
    UNION ALL
    SELECT *, _metadata.file_path AS archivo_fuente
    FROM volumen_ventas.default.ventas_curridabat
    UNION ALL
    SELECT *, _metadata.file_path AS archivo_fuente
    FROM volumen_ventas.default.ventas_heredia
    UNION ALL
    SELECT *, _metadata.file_path AS archivo_fuente
    FROM volumen_ventas.default.ventas_desamparados
    UNION ALL
    SELECT *, _metadata.file_path AS archivo_fuente
    FROM volumen_ventas.default.ventas_escazu
    UNION ALL
    SELECT *, _metadata.file_path AS archivo_fuente
    FROM volumen_ventas.default.ventas_guadalupe
    UNION ALL
    SELECT *, _metadata.file_path AS archivo_fuente
    FROM volumen_ventas.default.ventas_limon
    UNION ALL
    SELECT *, _metadata.file_path AS archivo_fuente
    FROM volumen_ventas.default.ventas_san_carlos
    UNION ALL
    SELECT *, _metadata.file_path AS archivo_fuente
    FROM volumen_ventas.default.ventas_puntarenas
    UNION ALL
    SELECT *, _metadata.file_path AS archivo_fuente
    FROM volumen_ventas.default.ventas_san_jose_centro
    UNION ALL
    SELECT *, _metadata.file_path AS archivo_fuente
    FROM volumen_ventas.default.ventas_san_pedro
    """
)

# ============================================================
# 2. Añadir columna de auditoría
# ============================================================
# Insertamos la columna 'fecha_ingesta' con la marca de tiempo
# en que se realiza la ingesta, para trazabilidad y auditoría.

df_TechMart = df_TechMart.withColumn("fecha_ingesta", current_timestamp())

# ============================================================
# 3. Escritura en la capa Bronze
# ============================================================
# Guardamos el DataFrame consolidado en formato Delta Lake.
# Usamos modo 'append' para permitir múltiples cargas incrementales.
# La tabla resultante se llama 'DF_TechMart_Bronze_Ingestion'.

df_TechMart.write.format("delta") \
    .mode("append") \
    .saveAsTable("DF_TechMart_Bronze_Ingestion")
